## Data Collection from user

##### Collect drowsy state video

In [1]:
# Disclaimer: Inform the user to act drowsy
print("Disclaimer: Please act drowsy for the next 10 seconds.")
#setting the disclaimer to 10 seconnds to allow the user to act drowsy

import cv2
import os
import time

time.sleep(1)
print("starting in 3")
time.sleep(1)
print("starting in 2")
time.sleep(1)
print("starting in 1")

def capture_video_and_save_frames(duration, output_folder):
    """
    Capture video for a given duration and save frames as JPEG images.
    
    Parameters:
        duration (int): Duration to capture video in seconds.
        output_folder (str): Path to save the captured frames.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the default camera
    cap = cv2.VideoCapture(0)

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Cannot open webcam.")
        return

    print(f"Recording video for {duration} seconds...")
    start_time = time.time()
    frame_count = 0

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # If frame is read correctly, ret is True
        if not ret:
            print("Error: Can't receive frame (stream end?). Exiting...")
            break

        # Save frame as JPEG file
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Stop capturing after the specified duration
        if time.time() - start_time > duration:
            break

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

# Record video of drowsy behavior (10 seconds)
capture_video_and_save_frames(10, 'drowsy_frames')


Disclaimer: Please act drowsy for the next 10 seconds.
starting in 3
starting in 2
starting in 1
Recording video for 10 seconds...


##### Collect Non Drowsy Sate Data

In [3]:
# Wait for 5 seconds
print("Please wait for 5 seconds.")
time.sleep(5)
def capture_video_and_save_frames1(duration, output_folder):
    """
    Capture video for a given duration and save frames as JPEG images.
    
    Parameters:
        duration (int): Duration to capture video in seconds.
        output_folder (str): Path to save the captured frames.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the default camera
    cap = cv2.VideoCapture(0)

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Cannot open webcam.")
        return

    print(f"Recording video for {duration} seconds...")
    start_time = time.time()
    frame_count = 0

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # If frame is read correctly, ret is True
        if not ret:
            print("Error: Can't receive frame (stream end?). Exiting...")
            break

        # Save frame as JPEG file
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Stop capturing after the specified duration
        if time.time() - start_time > duration:
            break

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

# Disclaimer: Inform the user to act normal
print("Please act normal for the next 10 seconds.")

# Record video of normal behavior (10 seconds)
capture_video_and_save_frames1(10, 'normal_frames')


Please wait for 5 seconds.
Please act normal for the next 10 seconds.
Recording video for 10 seconds...


##### Importing necessary libraries for facial feature detection and loading the facial detection

In [4]:
# la
from scipy.spatial import distance
import dlib

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate Mouth Aspect Ratio (MAR)
def calculate_mar(mouth):
    A = distance.euclidean(mouth[3], mouth[9])
    B = distance.euclidean(mouth[2], mouth[10])
    C = distance.euclidean(mouth[4], mouth[8])
    D = distance.euclidean(mouth[0], mouth[6])
    mar = (A + B + C) / (3.0 * D)
    return mar

# Load the facial landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Download required


##### extracting the features 


In [5]:
# Feature extraction code remains similar, reuse the EAR and MAR calculation functions
# Extract features from 'drowsy_frames' and 'normal_frames' folders.
# Save features as a .csv file for training later.
import numpy as np
import pandas as pd
import glob

def extract_features_from_frames(input_folder, label, output_csv):
    """
    Extract EAR, MAR, and other features from frames and save to CSV.
    """
    data = []
    files = sorted(glob.glob(os.path.join(input_folder, "*.jpg")))

    for file in files:
        frame = cv2.imread(file)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)
            left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
            right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
            mouth = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)])

            ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0
            mar = calculate_mar(mouth)
            data.append([ear, mar, label])

    # Save data to CSV
    df = pd.DataFrame(data, columns=['EAR', 'MAR', 'Label'])
    df.to_csv(output_csv, index=False)

# Extract features for both classes
extract_features_from_frames('drowsy_frames', 1, 'drowsy_features.csv')
extract_features_from_frames('normal_frames', 0, 'normal_features.csv')


##### Training the model

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Load data
drowsy_data = pd.read_csv('drowsy_features.csv')
normal_data = pd.read_csv('normal_features.csv')
data = pd.concat([drowsy_data, normal_data])

X = data[['EAR', 'MAR']].values
y = data['Label'].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a lightweight neural network
model = Sequential([
    Dense(8, input_shape=(2,), activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7251 - loss: 0.6428 - val_accuracy: 0.7373 - val_loss: 0.6249
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7790 - loss: 0.5912 - val_accuracy: 0.7458 - val_loss: 0.5958
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7946 - loss: 0.5668 - val_accuracy: 0.7458 - val_loss: 0.5712
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 0.7879 - loss: 0.5392 - val_accuracy: 0.7627 - val_loss: 0.5443
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.7996 - loss: 0.5126 - val_accuracy: 0.7712 - val_loss: 0.5174
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - accuracy: 0.7961 - loss: 0.4782 - val_accuracy: 0.7797 - val_loss: 0.4958
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.8379 - loss: 0.4365 - val_accuracy: 0.7881 - val_loss: 0.4785
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - accuracy: 0.8274 - loss: 0.4258 - val_accuracy: 0.7966 - val_loss: 

##### Converting to tflite code and optimising it out

In [7]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open('drowsiness_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved as drowsiness_model.tflite")


INFO:tensorflow:Assets written to: /var/folders/5d/cxg576h92j906wv9p0xyjc3c0000gn/T/tmp3y2lrokq/assets


INFO:tensorflow:Assets written to: /var/folders/5d/cxg576h92j906wv9p0xyjc3c0000gn/T/tmp3y2lrokq/assets


Saved artifact at '/var/folders/5d/cxg576h92j906wv9p0xyjc3c0000gn/T/tmp3y2lrokq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  12146675152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12198575952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12198575568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12198574416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12203147920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12203148496: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model saved as drowsiness_model.tflite


W0000 00:00:1731582251.263766   38143 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1731582251.263816   38143 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-11-14 16:34:11.264053: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/5d/cxg576h92j906wv9p0xyjc3c0000gn/T/tmp3y2lrokq
2024-11-14 16:34:11.264399: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-11-14 16:34:11.264405: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/5d/cxg576h92j906wv9p0xyjc3c0000gn/T/tmp3y2lrokq
I0000 00:00:1731582251.267335   38143 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-11-14 16:34:11.267835: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-11-14 16:34:11.286802: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folder